In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)

from sklearn.model_selection import train_test_split

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

import numpy as np
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier, MLPRegressor
from collections import Counter


In [2]:
property_df = pd.read_csv("../data/curated/full_data.csv")

In [3]:
property_df.head(1)

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.0,5,3,3,House,mud brick magic,211021262,145.209992,-37.747366,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262.0


In [4]:
X = property_df
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
single_counts = X2[X2['count']==1].reset_index()['SA2_NAME21'].tolist() # can improve this later with bootstrapping
X = X[~X["SA2_NAME21"].isin(single_counts)]
X.head(1)

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.0,5,3,3,House,mud brick magic,211021262,145.209992,-37.747366,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262.0


In [5]:
y = X['cost_text'].tolist()
type(y[0])
# note: dropping desc head for now since it is not valuable yet
# SA2 CODE covered by SA2 Name
X.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", "cost_text"], axis=1, 
       inplace=True)
X.astype({ 
          "Median_tot_prsnl_inc_weekly":"float", 
          "Median_rent_weekly":"float",
          "Median_tot_fam_inc_weekly":"float", 
          "Median_tot_hhd_inc_weekly":"float", 
          "total_population": "int64",
          "duration_mins": "category", 
          "Bed":"int32", 
          "Bath":"int32", 
          "Park":"int32",
          "property_type":"category",
          "SA2_NAME21": "category",
          "Offence Count": "int32"}).dtypes

l = [(elem, type(elem)) for elem in X.loc[0]]
print(l)
X.head(1)

[(5, <class 'numpy.int64'>), (3, <class 'numpy.int64'>), (3, <class 'numpy.int64'>), ('House', <class 'str'>), (930.0, <class 'numpy.float64'>), (481.0, <class 'numpy.float64'>), (3051.0, <class 'numpy.float64'>), (2779.0, <class 'numpy.float64'>), (8781.0, <class 'numpy.float64'>), ('Warrandyte - Wonga Park', <class 'str'>), ('>50.0', <class 'str'>), (859.0, <class 'numpy.float64'>)]


/tmp/ipykernel_16643/3736008520.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", "cost_text"], axis=1,


,Bed,Bath,Park,property_type,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count
0,5,3,3,House,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0


In [6]:
type(y[0])

float

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
scaler = StandardScaler()
from sklearn.compose import ColumnTransformer

numeric_features = [ 
                  "Median_tot_prsnl_inc_weekly", 
                  "Median_rent_weekly",
                  "Median_tot_fam_inc_weekly", 
                  "Median_tot_hhd_inc_weekly", 
                  "total_population",
                  "Bed", 
                  "Bath", 
                  "Park",
                  "Offence Count"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_features)])

clf = MLPRegressor(max_iter=5000)
#this way we don't cheat. Read more on pipelines https://scikit-learn.org/stable/modules/compose.html
pipeline = Pipeline([('transformer', preprocessor), ('estimator', clf)]) # ('reduce_dim', PCA()),



In [ ]:
print('Cross-val standardised features R^2:', np.mean(cross_val_score(pipeline, X, y, cv=10)))
# 36%... not good. might need to perform PCA to improve, and further feature selection.
# can try grid search to tune hyperparams. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1, stratify=X[['SA2_NAME21']])


clf.fit(preprocessor.fit_transform(X_train),y_train) # need to perform a train test split here
# need to do balanced split- i.e. one of the bootstrapping methods or something to make sure all suburbs have
# a good amount of representation. 


#print(clf.coefs_)
print('parameter shapes:',[p.shape for p in clf.coefs_])
print('num layers:', clf.n_layers_)
print('R^2 these params:', clf.score(preprocessor.transform(X_test), y_test)) # 34%


In [ ]:
from sklearn.metrics import mean_squared_error
y_pred = clf.predict(preprocessor.transform(X_test))
print(mean_squared_error(y_test, y_pred))

## Some Notes
So we see that we only get an R^2 of 34%. This is before tuning hyperparams, but even then, it is difficult to believe changing hyperparameters will be a significant enough boost. Let us try another method. 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

alphas = [np.power(10.0, i) for i in range(-7, 2)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, 
                                                          stratify=X[['SA2_NAME21']])


clf.fit(preprocessor.fit_transform(X_train), y_train)
print('MLP R^2 without tuning:', clf.score(preprocessor.transform(X_test), y_test))

hidden_sizes = [[220], [550, 220]]
#arguments of MLPClassifier and a list of values for them to search and find the best.
param_grid = {'alpha': alphas, 'hidden_layer_sizes':hidden_sizes}

# Make an appropriate scoring function

gs = GridSearchCV(estimator=clf,
                  param_grid=param_grid,
                  scoring='neg_mean_squared_error',
                  cv=3,
                  n_jobs=1, # Somehow verbose will not print if you do multithread
                  verbose=1 # More verbose = More detailed print
                  )

gs.fit(preprocessor.fit_transform(X_train), y_train)
best_params = gs.best_params_
print('best_params', best_params)

clf = MLPRegressor(max_iter=5000, **best_params)
clf.fit(preprocessor.fit_transform(X_train), y_train)
print('R^2 with best params:', clf.score(preprocessor.transform(X_test), y_test))

In [ ]:


import pandas as pd
pd.DataFrame([gs.cv_results_['param_alpha'],
              gs.cv_results_['param_hidden_layer_sizes'],
              gs.cv_results_['mean_test_score'],
              gs.cv_results_['mean_fit_time'],
              gs.cv_results_['rank_test_score']], 
             index=['alpha','Hidden layer size','Mean test score', 'Mean fit time', 'Ranking'])



We see that the accuracy is slightly low, despite our hypter parameter tuning. It is likely that the sheer amount of data is making it difficult to build a good model. We could try dividing things into categories and building seperate models. 

In [8]:
import geopandas as gpd
# sf stands for shape file
australia_sf = gpd.read_file("../data/raw/shapefiles/Statistical_area_level2/SA2_2021_AUST_GDA2020.shp")
australia_sf.head(3)
# seperate suburbs that are in Victoria
vic_sf = australia_sf[australia_sf['STE_NAME21'] == 'Victoria']

# drop the null location ID values
vic_sf.dropna(inplace=True)
vic_sf.head(1)

/tmp/ipykernel_16643/2689031410.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vic_sf.dropna(inplace=True)


,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
644,201011001,Alfredton,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,52.7109,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5..."


In [9]:
vic_centroid_sf = vic_sf
vic_centroid_sf['geometry'] = vic_sf['geometry'].to_crs('+proj=cea').centroid.to_crs(vic_sf['geometry'].crs)
cbd_loc = vic_centroid_sf.where(vic_centroid_sf['SA2_NAME21'] == 'Melbourne CBD - North').dropna()['geometry']
cbd_loc

/home/sesshah/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


780    POINT (144.96089 -37.80918)
Name: geometry, dtype: geometry

In [10]:
# function taken from https://stackoverflow.com/a/56661833

from math import radians, cos, sin, asin, sqrt, inf

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# this function converts two floats to a point that is recognised by Shapely
def point_to_dist(row):
    dist = haversine(row["geometry"].y, row["geometry"].x, cbd_loc.y, cbd_loc.x)
    return dist


In [33]:
X = property_df
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
single_counts = X2[X2['count']==1].reset_index()['SA2_NAME21'].tolist() # can improve this later with bootstrapping
X = X[~X["SA2_NAME21"].isin(single_counts)]
X2 = X[['SA2_NAME21']].groupby(by="SA2_NAME21")["SA2_NAME21"].count()
X2 = pd.DataFrame(X2)
X2.columns=["count"]
X3 = vic_centroid_sf.join(X2, how='inner',on='SA2_NAME21')


X3['cbd_dist'] = X3.apply (lambda row: point_to_dist(row), axis=1)
X3["SA2_CODE21"] = X3["SA2_CODE21"].astype(int)

In [34]:
X3.sort_values(by=['cbd_dist'], inplace=True)
X3.set_index('SA2_CODE21', inplace=True)
X3

,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry,count,cbd_dist
SA2_CODE21,,,,,,,,,,,,,,,,,,
206041504,Melbourne CBD - North,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.5616,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96089 -37.80918),252,0.000000
206041505,Melbourne CBD - West,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,1.0051,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.95871 -37.81672),200,0.728392
206041503,Melbourne CBD - East,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.8020,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96860 -37.81277),138,0.917602
206041117,Carlton,0,No change,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,1.8187,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96781 -37.80049),218,1.103787
206041510,West Melbourne - Residential,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.8247,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.94977 -37.80857),67,1.237667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212031557,Cranbourne East - North,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,11.0254,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.31291 -38.10222),12,47.402490
212021293,Berwick - North,0,No change,21202,Casey - North,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,19.7480,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.34885 -38.02511),32,47.426478
212031558,Cranbourne East - South,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,5.3272,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.30423 -38.12247),13,47.691723


In [35]:
property_df['SA2_CODE_2021'] = property_df['SA2_CODE_2021'].astype(int)
property_df

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,latitude,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.000000,5,3,3,House,mud brick magic,211021262,145.209992,-37.747366,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262
1,https://www.domain.com.au/6-airlie-court-donca...,6 Airlie Court Doncaster East VIC 3109,850.000000,3,2,1,House,spaciou home east,207021424,145.150927,-37.777456,704.0,497.0,2158.0,1934.0,13507.0,Doncaster East - North,>50.0,859.0,207021424
2,https://www.domain.com.au/6-halcyon-court-donc...,6 HALCYON COURT Doncaster East VIC 3109,830.000000,4,2,2,House,spaciou four bedroom plu studi home view east,207021424,145.166327,-37.775254,704.0,497.0,2158.0,1934.0,13507.0,Doncaster East - North,>50.0,859.0,207021424
3,https://www.domain.com.au/7-davy-court-narre-w...,7 Davy Court Narre Warren South VIC 3805,795.000000,4,2,2,House,bedroom famili home prime locat warren south,212031457,145.318043,-38.061485,767.0,430.0,2350.0,2352.0,10833.0,Narre Warren South - East,>50.0,859.0,212031457
4,https://www.domain.com.au/level-1-301-13-15-la...,"Level 1, 301/13-15 Lake Street Caroline Spring...",793.269231,4,3,2,Apartment / Unit / Flat,endless possibl,213041463,144.744689,-37.731646,813.0,401.0,2277.0,2133.0,16696.0,Caroline Springs,>50.0,859.0,213041463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,https://www.domain.com.au/8-298-nepean-highway...,8/298 Nepean Highway Seaford VIC 3198,365.000000,2,1,2,Apartment / Unit / Flat,easi care lifestyl,214011375,145.124349,-38.128700,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),20.0,1018.0,214011375
10860,https://www.domain.com.au/8-90-nepean-highway-...,8/90 Nepean Highway Seaford VIC 3198,300.000000,2,1,1,Apartment / Unit / Flat,beach side locat,214011375,145.126181,-38.098008,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),20.0,1018.0,214011375
10861,https://www.domain.com.au/86-centenary-street-...,86 Centenary Street Seaford VIC 3198,480.000000,4,2,3,House,solid bedroom home plenti space,214011375,145.158473,-38.110641,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),40.0,1018.0,214011375
10862,https://www.domain.com.au/9-a-ti-tree-crescent...,9 A Ti Tree Crescent Seaford VIC 3198,285.000000,2,1,1,Apartment / Unit / Flat,privat rear courtyard bungalow plu garden bdrm...,214011375,145.156094,-38.111442,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),40.0,1018.0,214011375


In [44]:

#property_df.set_index('SA2_CODE21')
X_dists= property_df.join(X3[['cbd_dist']], how='inner', on='SA2_CODE_2021')

In [45]:
X_dists

,index,address,cost_text,Bed,Bath,Park,property_type,desc_head,LocID,longitude,...,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21,duration_mins,Offence Count,SA2_CODE_2021,cbd_dist
0,https://www.domain.com.au/warrandyte-vic-3113-...,Warrandyte VIC 3113,875.0,5,3,3,House,mud brick magic,211021262,145.209992,...,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262,31.931748
20,https://www.domain.com.au/389a-heidelberg-warr...,389A Heidelberg-Warrandyte Road Warrandyte VIC...,670.0,3,2,2,House,nativ retreat,211021262,145.204065,...,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262,31.931748
177,https://www.domain.com.au/9-314a-reynolds-road...,9/314A Reynolds Road Donvale VIC 3111,420.0,2,1,1,House,mapl rise independ live year,211021262,145.190657,...,930.0,481.0,3051.0,2779.0,8781.0,Warrandyte - Wonga Park,>50.0,859.0,211021262,31.931748
1,https://www.domain.com.au/6-airlie-court-donca...,6 Airlie Court Doncaster East VIC 3109,850.0,3,2,1,House,spaciou home east,207021424,145.150927,...,704.0,497.0,2158.0,1934.0,13507.0,Doncaster East - North,>50.0,859.0,207021424,23.452476
2,https://www.domain.com.au/6-halcyon-court-donc...,6 HALCYON COURT Doncaster East VIC 3109,830.0,4,2,2,House,spaciou four bedroom plu studi home view east,207021424,145.166327,...,704.0,497.0,2158.0,1934.0,13507.0,Doncaster East - North,>50.0,859.0,207021424,23.452476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,https://www.domain.com.au/8-298-nepean-highway...,8/298 Nepean Highway Seaford VIC 3198,365.0,2,1,2,Apartment / Unit / Flat,easi care lifestyl,214011375,145.124349,...,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),20.0,1018.0,214011375,33.402832
10860,https://www.domain.com.au/8-90-nepean-highway-...,8/90 Nepean Highway Seaford VIC 3198,300.0,2,1,1,Apartment / Unit / Flat,beach side locat,214011375,145.126181,...,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),20.0,1018.0,214011375,33.402832
10861,https://www.domain.com.au/86-centenary-street-...,86 Centenary Street Seaford VIC 3198,480.0,4,2,3,House,solid bedroom home plenti space,214011375,145.158473,...,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),40.0,1018.0,214011375,33.402832
10862,https://www.domain.com.au/9-a-ti-tree-crescent...,9 A Ti Tree Crescent Seaford VIC 3198,285.0,2,1,1,Apartment / Unit / Flat,privat rear courtyard bungalow plu garden bdrm...,214011375,145.156094,...,809.0,351.0,2081.0,1500.0,15258.0,Seaford (Vic.),40.0,1018.0,214011375,33.402832


In [46]:
count = 0
BREAK = 2500
li = []
suburb_lists = []
for index, row in X3.iterrows():
    #print(row)
    count += int(row['count'])
    li.append(row["SA2_NAME21"])
    if len(suburb_lists) < 3: 
        if count >= BREAK:
            count = 0
            suburb_lists.append(li)
            li = []
suburb_lists.append(li)

In [49]:
sl = 100
models = []
numeric_features = [ 
                  "Median_tot_prsnl_inc_weekly", 
                  "Median_rent_weekly",
                  "Median_tot_fam_inc_weekly", 
                  "Median_tot_hhd_inc_weekly", 
                  "total_population",
                  "Bed", 
                  "Bath", 
                  "Park",
                  "Offence Count",
                  "cbd_dist"]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numeric_features)])

for i in range(len(suburb_lists)): 
    print(f"\n-- Beginning Ring {i} -- ")
    suburbs = suburb_lists[i]
    test_df = X_dists.where(X_dists['SA2_NAME21'].isin(suburbs)).dropna()

    y = test_df['cost_text'].tolist()
    print("\tnum training instances: ", len(y))
    # note: dropping desc head for now since it is not valuable yet
    # SA2 CODE covered by SA2 Name
    test_df.drop(["index", "address", "desc_head", "latitude", "longitude", "LocID", "SA2_CODE_2021", 
                  "cost_text", "SA2_NAME21"], axis=1, 
           inplace=True)
    test_df.astype({ 
              "Median_tot_prsnl_inc_weekly":"float", 
              "Median_rent_weekly":"float",
              "Median_tot_fam_inc_weekly":"float", 
              "Median_tot_hhd_inc_weekly":"float", 
              "total_population": "int64",
              "duration_mins": "category", 
              "Bed":"int32", 
              "Bath":"int32", 
              "Park":"int32",
              "property_type":"category",
              #"SA2_NAME21": "category",
              "Offence Count": "int32",
              "cbd_dist": "float"}).dtypes

    X_train, X_test, y_train, y_test = train_test_split(test_df, y, random_state=0, test_size=0.1)
                                                        #stratify=test_df[['SA2_NAME21']])
    clf = MLPRegressor(max_iter=10000,hidden_layer_sizes=[sl])

    clf.fit(preprocessor.fit_transform(X_train),y_train)  

    models.append(clf)
    #print(clf.coefs_)
    print('\tparameter shapes:',[p.shape for p in clf.coefs_])
    print('\tnum layers:', clf.n_layers_)
    print('\tR^2 these params:', clf.score(preprocessor.transform(X_test), y_test)) 
    # 0.36, 0.44, 0.45, 0.51 R^2s- better. 


-- Beginning Ring 0 -- 
	num training instances:  2573
	parameter shapes: [(10, 100), (100, 1)]
	num layers: 3
	R^2 these params: 0.233810455273036

-- Beginning Ring 1 -- 
	num training instances:  2498
	parameter shapes: [(10, 100), (100, 1)]
	num layers: 3
	R^2 these params: 0.398515433482283

-- Beginning Ring 2 -- 
	num training instances:  2505
	parameter shapes: [(10, 100), (100, 1)]
	num layers: 3
	R^2 these params: 0.5049287807390557

-- Beginning Ring 3 -- 
	num training instances:  3264
	parameter shapes: [(10, 100), (100, 1)]
	num layers: 3
	R^2 these params: 0.5110137822800109


In [56]:
X_dists.head(1)
categorical_features = ["property_type","duration_mins" ]

In [57]:

cur_avg_rents = X_dists.groupby(['SA2_CODE_2021'])['cost_text'].mean()
cur_avg_rents

SA2_CODE_2021
206011106    502.875000
206011107    453.350000
206011109    487.678571
206011495    550.742706
206011496    548.450704
                ...    
214011372    439.687500
214011373    593.181818
214011374    497.222222
214011375    448.454545
214011376    610.000000
Name: cost_text, Length: 249, dtype: float64

In [86]:
e = pd.DataFrame(X_dists[numeric_features].mean()).transpose()
ee = pd.DataFrame(X_dists[categorical_features].mode())
e['suburb'] = 'steve'
ee['suburb'] = 'steve'
e.set_index('suburb', inplace=True)
ee.set_index('suburb', inplace=True)
avg_house = e.join(ee, how='inner')
avg_house


,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,Bed,Bath,Park,Offence Count,cbd_dist,property_type,duration_mins
suburb,,,,,,,,,,,,
steve,927.08232,399.209392,2419.245028,1870.794936,13827.733241,2.462799,1.41593,1.271639,981.233241,13.264794,Apartment / Unit / Flat,10.0


In [100]:
X3

,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry,count,cbd_dist
SA2_CODE21,,,,,,,,,,,,,,,,,,
206041504,Melbourne CBD - North,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.5616,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96089 -37.80918),252,0.000000
206041505,Melbourne CBD - West,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,1.0051,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.95871 -37.81672),200,0.728392
206041503,Melbourne CBD - East,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.8020,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96860 -37.81277),138,0.917602
206041117,Carlton,0,No change,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,1.8187,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.96781 -37.80049),218,1.103787
206041510,West Melbourne - Residential,1,New,20604,Melbourne City,206,Melbourne - Inner,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,0.8247,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (144.94977 -37.80857),67,1.237667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212031557,Cranbourne East - North,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,11.0254,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.31291 -38.10222),12,47.402490
212021293,Berwick - North,0,No change,21202,Casey - North,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,19.7480,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.34885 -38.02511),32,47.426478
212031558,Cranbourne East - South,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,5.3272,http://linked.data.gov.au/dataset/asgsed3/SA2/...,POINT (145.30423 -38.12247),13,47.691723


In [105]:
df_lists = pd.DataFrame([])
for index, row in X3.iterrows():
    cur_sub = index
    test_df = X_dists.where(X_dists['SA2_CODE_2021'] == cur_sub).dropna()
    df_num = pd.DataFrame(test_df[numeric_features].mean()).transpose()
    df_cat = pd.DataFrame(test_df[categorical_features].mode())
    df_num['suburb'] = cur_sub
    df_cat['suburb'] = cur_sub
    df_num.set_index('suburb', inplace=True)
    df_cat.set_index('suburb', inplace=True)
    avg_house = df_cat.join(df_num, how='inner')
    if df_lists.empty:
        df_lists = avg_house
    else:
        df_lists = pd.concat([df_lists, avg_house])

In [107]:
df_lists.dropna(inplace=True) # i shouldnt have NAs though. 

In [109]:
df_lists # still 4 more than there should be...

,property_type,duration_mins,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,Bed,Bath,Park,Offence Count,cbd_dist
suburb,,,,,,,,,,,,
206041504,Apartment / Unit / Flat,10.0,638.0,361.0,1520.0,1089.0,17891.0,1.658730,1.329365,0.250000,1073.0,0.000000
206041505,Apartment / Unit / Flat,10.0,861.0,376.0,1945.0,1446.0,16491.0,1.620000,1.340000,0.405000,859.0,0.728392
206041503,Apartment / Unit / Flat,10.0,884.0,375.0,2248.0,1511.0,10533.0,1.681159,1.231884,0.427536,859.0,0.917602
206041117,Apartment / Unit / Flat,20.0,652.0,365.0,2064.0,1289.0,17080.0,1.839450,1.220183,0.394495,926.0,1.103787
206041510,Apartment / Unit / Flat,10.0,1047.0,390.0,2397.0,1784.0,7460.0,1.692308,1.261538,0.692308,428.0,1.237667
...,...,...,...,...,...,...,...,...,...,...,...,...
212031557,House,40.0,792.0,400.0,1985.0,1931.0,17852.0,3.666667,2.083333,1.833333,859.0,47.402490
212021293,House,30.0,820.0,380.0,2311.0,1885.0,20218.0,3.225806,1.774194,1.741935,1105.0,47.426478
212031558,House,50.0,855.0,401.0,2036.0,1958.0,9482.0,3.769231,1.923077,2.000000,859.0,47.691723
